# Generic

In [1]:
import hashlib, base64
def sha256(msg):
    hasher = hashlib.sha256()
    hasher.update(msg.encode())
    return hasher.hexdigest()
def sha1(msg):
    hasher = hashlib.sha1()
    hasher.update(msg.encode())
    return hasher.hexdigest()
def sha1i(msg):
    """SHA1 but it returns an integer"""
    return int(sha1(msg), 16)
def bytes_to_int(b):
    return int.from_bytes(b, 'big')
def int_to_bytes(i):
    s = hex(i)[2:].upper()
    if len(s) % 2 == 1:
        s = '0' + s
    return base64.b16decode(s)

# Crypto

In [2]:
# First, get some primitives down :

import gensafeprime

def primegen(small=True, big=False):
    """Generate a random prime."""
    if small:
        return gensafeprime.generate(32)
    if big:
        return gensafeprime.generate(1024)
    return gensafeprime.generate(128)

def egcd(r, u, v, rp, up, vp):
    """Extended Euclid algorithm for GCD
    Given a, b, find GCD(a,b) and x,y such that :
        a.x b+ b.y = GCD(a,b)     (Bézout coefficients)
    """
    if r == 0:
        return up, vp
    else:
        return egcd(rp - (rp // r)*r, up - (rp // r)*u, vp - (rp // r)*v, r, u, v)
    
    
def invmod(a, N):
    """Calculate the inverse modulo"""
    x, _ = egcd(a, 1, 0, N, 0, 1)
    return x % N

assert (invmod(15, 37) * 15) % 37 == 1

In [3]:
import base64

def str_to_int(s):
    return int(base64.b16encode(s), 16)
def int_to_str(i):
    s = hex(i)[2:].upper()
    if len(s) % 2 == 1:
        s = '0' + s # base16 wants even length strings, adding a 0 in front does not change the value. Perfect.
    return base64.b16decode(s)
assert int_to_str(str_to_int((b"Hello, world !"))) == b'Hello, world !'

class RSA:
    def __init__(self, small=True, big=False, block_length=None, p=None, q=None):
        if p is None:
            self.p = primegen(small=small, big=big)
        else:
            self.p = p
        if q is None:
            self.q = primegen(small=small, big=big)
        else:
            self.q = q
        self.n = self.p * self.q
        self.et = (self.p - 1) * (self.q - 1)
        self.e = 3
        self.d = invmod(self.e, self.et)
        if block_length is None:
            self.block_length = 3 + (not small)*9
        else:
            self.block_length = block_length
    
    @property
    def public(self):
        return self.e, self.n
    
    @property
    def private(self):
        return self.d, self.n
    
    def encrypt_block(self, s, i2s = True):
        # Encrypt one block
        if isinstance(s, bytes):
            m = str_to_int(s)
            if m ** 3 < self.n:
                print('Unsafe encryption.') # If the cubed message does not wrap around, it's not much use...
            if m > self.n:
                raise Exception("Message not encryptable. Please use blocks.")
            if i2s:
                return int_to_str(pow(m, self.e, self.n))
            else:
                return pow(m, self.e, self.n)
        else:
            m = s
            return pow(m, self.e, self.n)
    
    def decrypt_block(self, s, i2s = True):
        if isinstance(s, bytes):
            c = str_to_int(s)
            if c > self.n:
                raise Exception("Message not encryptable. Please use blocks.")
            if i2s:
                return int_to_str(pow(c, self.d, self.n))
            else:
                return pow(c, self.d, self.n)
        else:
            c = s
            return pow(c, self.d, self.n)
    
    def encrypt(self, s):
        acc = []
        for i in range(0, len(s), self.block_length):
            acc.append(self.encrypt_block(s[i:i+self.block_length]))
        return acc
        
    
    def decrypt(self, s):
        return b"".join(self.decrypt_block(b) for b in s)
        
    
r = RSA(small=False)
assert r.decrypt(r.encrypt(b"Vanilla Ice Baby")) == b'Vanilla Ice Baby'

Unsafe encryption.


# Cryptopals Notebook

## Set 6

### Exercice 41

Implement unpadded message recovery oracle

In [4]:
# Setting up legitimate side of things
class Server:
    def __init__(self):
        self.rsa = RSA(small=False)
        self.memory = set()
    
    def encrypt(self, s):
        return self.rsa.encrypt(s)
    
    def decrypt(self, s):
        if b"".join(s) in self.memory:
            raise Exception("Message already decrypted") # Why bother with hashes
        else:
            self.memory.add(b"".join(s))
            return self.rsa.decrypt(s)

# Alice is the server
alice = Server()

# Bob is the JS client
original_message = b'{ "time": 62365314455, "social": "555-555-555" }'
bob = {'message': alice.encrypt(original_message) }

# Bob asks for the decrypted message
bob['clear'] = alice.decrypt(bob['message'])
assert bob['clear'] == original_message

In [5]:
# Eve gets a hold of the ciphertext !
eve = {'message': bob['message']}

# Ensure that the server won't decrypt it again...
try:
    alice.decrypt(eve['message'])
    assert False
except:
    assert True
    
# Now modify message so that is different but still the same.
(e, n), s = alice.rsa.public, 4 # Totally random, I swear.
c = [str_to_int(block) for block in eve['message']] # Go back to ints for math
cp = [(pow(s, e, n) * block) % n for block in c]
assert c != cp
new_message = alice.decrypt([int_to_str(block) for block in cp])
assert new_message != original_message # We got a jumbled mess here

In [6]:
# Now Eve will want to recover original_message from new_message :
blocks = [str_to_int(new_message[i:i+12]) for i in range(0, len(new_message), 12)]
new_blocks = [(block * invmod(s, n)) % n for block in blocks]
decrypted_message = b"".join(int_to_str(block) for block in new_blocks)

# This is due to the fact that we add 0's to ensure int_to_str. Not that important.
# We could just make a smarter way to convert str to ints. Meh.
decrypted_message = b"".join(bytes([b]) for b in decrypted_message if b != 0)
assert decrypted_message == original_message

### Exercice 42

Bleichenbacher's e=3 RSA Attack.

In [7]:
sender = RSA(small=False, big=True)

In [8]:
def sign(sender, text):
    """Simple implementation of RSA signing"""
    d, n = sender.private
    _hash = base64.b16decode(sha1(text).upper())
    asn1 = b'\x30\x21\x30\x09\x06\x05\x2b\x0e\x03\x02\x1a\x05\x00\x04\x14' # ASN.1 encoding for SHA1
    block = b'\x00\x01' + b'\xff' * (128 - len(_hash) - len(asn1) - 3) + b'\x00' + asn1 + _hash # Pad out to 128 bytes
    assert len(block) == 128 # 1024 bit blocks
    return pow(bytes_to_int(block), d, n)

def check_signature(sender, text, signature):
    """Flawed implementation of RSA signature verification"""
    e, n = sender.public
    _hash = base64.b16decode(sha1(text).upper())
    asn1 = b'\x30\x21\x30\x09\x06\x05\x2b\x0e\x03\x02\x1a\x05\x00\x04\x14' # ASN.1 encoding for SHA1
    clearint = pow(signature, e, n)
    clearblock = b'\x00' + int_to_bytes(clearint) # Initial null byte is lost during integer conversion
    if not (clearblock[0:3] == b'\x00\x01\xff'):
        print('Starting bytes not ok')
        return False # Check only the "important" bytes
    clearblock = clearblock[3:]
    while clearblock[0] == 255:
        clearblock = clearblock[1:] # Consume 0xFF bytes
    return clearblock.startswith(b'\x00' + asn1 + _hash) # Check important bytes
    # So we did NOT check that the padding was the entire length
    # and we did NOT check the remaining block was equal to the ans1 + hash part.
    
# Check base case
assert check_signature(sender, "Hello, world !", sign(sender, "Hello, world !"))

This is all inspired from this submission : https://www.ietf.org/mail-archive/web/openpgp/current/msg00999.html

Let's define D and N :
$$\begin{align*}
D
&= 0x00\ ASN.1\ HASH \\
&= 2^{288} - N
\end{align*}$$

And now build the signature, using the least amount of padding possible (to have the largest possible margin) :
$$\begin{align*}
signature
&= 0x00\ 0x01\ 0xFF\ 0x00\ ASN.1\ HASH\ GARBAGE\ (1024\ bit\ key)\\
&= 2^{1009} - 2^{1000} + D.2^{712} + garbage\\
&= 2^{1009} - 2^{1000} + 2^{1000} - N.2^{712} + garbage\\
&= 2^{1009} - N.2^{712} + garbage\\
\end{align*}$$

Sadly, given that 2^1009 is not a perfect cube, we cannot make a nice formula like in the Hal Finney writeup. We still have a good base. Let's crunch some numbers to find a perfect cube that would fit our description. WolframAlpha reveals that the cube root of our baseline (2^1009 - N\*2^712) is around 1.7\*10^101. Let's use that as a starting value (note: we could have found this value by using the same binary search used to find the perfect root).

In [9]:
# Now, we want to forge a signature
e, n = sender.public
_hash = base64.b16decode(sha1('Hello, world !').upper())
asn1 = b'\x30\x21\x30\x09\x06\x05\x2b\x0e\x03\x02\x1a\x05\x00\x04\x14' # ASN.1 encoding for SHA1
D = bytes_to_int(b'\x00' + asn1 + _hash)
N = 2**288 - D
target = 2**1009 - N * (2**712)
base = 176254032823638680266052181223788664320230109282117832543013801901130780612858290399771331518246797307

In [10]:
# We need a perfect cube that is slightly larger than our target (less than 2^712 difference, should be doable)
def is_good_root(n):
    if (n**3 - target) < 0:
        return -1 # Too small
    if (n**3 - target) > 2**712:
        return 1  # Too big
    return 0 # Perfect.

low = base
high = base*2
mid = (high - low) // 2 + low
is_ok = is_good_root(mid)
while is_ok != 0:
    if is_ok < 0:
        low = mid
    elif is_ok > 0:
        high = mid
    mid = (high - low) // 2 + low
    is_ok = is_good_root(mid)
root = mid
    
print('Found a good root. (root^3 - target) is %s and (2^712 - (root^3 - target)) is %s' % (mid**3 - target, 2**712 - (mid**3 - target)))

Found a good root. (root^3 - target) is 16127599852352072733114186255968266580760613075092633068618640212423478497054982267802592076314564076913940744080994100546246971644854173908828995096951797527509070032754050928806580131027491716868344335397286586176 and (2^712 - (root^3 - target)) is 5417916800390065152544908304308740433333578757271087059886318366546048406945860779574986056278438219311781496353911879206280622851898686714542534203638306433907526124188058145386474621266694133074872823712473929920


In [11]:
# Now for the test !
assert check_signature(sender, "Hello, world !", root)

# Interlude : Cyclic Groups Math

In [12]:
class CyclicInteger:
    def __init__(self, value, n):
        self.value = int(value)
        self.n = int(n)
    def __int__(self):
        return int(self.value) % int(self.n)
    def __add__(self, i):
        return CyclicInteger((int(self) + int(i)) % self.n, self.n)
    def __sub__(self, i):
        return CyclicInteger((int(self) - int(i)) % self.n, self.n)
    def __eq__(self, i):
        return int(self) == int(i) % self.n
    def __lt__(self, i):
        return int(self) < int(i) % self.n
    def __gt__(self, i):
        return int(self) > int(i) % self.n
    def __mul__(self, i):
        return CyclicInteger((int(self)*int(i)) % self.n, self.n)
    def __truediv__(self, i):
        return CyclicInteger((int(self) * invmod(int(i), self.n)) % self.n, self.n)
    def __pow__(self, i, m=None):
        if m is None:
            return CyclicInteger(pow(int(self), int(i), self.n), self.n)
        else:
            return CyclicInteger(pow(int(self), int(i), self.n) % int(m), self.n)
    def __mod__(self, i):
        return CyclicInteger((int(self) % int(i)) % self.n, self.n)
    @property
    def inverse(self):
        return CyclicInteger(invmod(int(self), self.n), self.n)
    
a, b, c = CyclicInteger(1, 13), CyclicInteger(10, 13), CyclicInteger(11, 13)
assert int(a - b) == 4  
assert int(b + c) == 8
assert int(c + 5) == 3
assert int(c * c.inverse) == a == 1
assert c.inverse.inverse == c
assert int(c / c.inverse.inverse) == a == 1
assert int(b ** 2) == 9

### Exercice 43

DSA key recovery from nonce

In [13]:
# Parameters
p = 0x800000000000000089e1855218a0e7dac38136ffafa72eda7859f2171e25e65eac698c1702578b07dc2a1076da241c76c62d374d8389ea5aeffd3226a0530cc565f3bf6b50929139ebeac04f48c3c84afb796d61e5a4f9a8fda812ab59494232c7d2b4deb50aa18ee9e132bfa85ac4374d7f9091abc3d015efc871a584471bb1
q = 0xf4f47f05794b256174bba6e9b396a7707e563c5b
g = 0x5958c9d3898b224b12672c0b98e06c60df923cb8bc999d119458fef538b8fa4046c8db53039db620c094c9fa077ef389b5322a559946a71903f990f1f7e0e025e2d7f7cf494aff1a0470f5b64c36b625a097f1651fe775323556fe00b3608c887892878480e99041be601a62166ca6894bdd41a7054ec89f756ba9fc95302291

In [14]:
import random

def dsa_sign(text, p, q, g, return_private_key=False, nonce=None, force_private_key=None,
            check_r_non_zero=True):
    """Sign a given text using DSA
    
    Note : some arguments are useful for later exercices.
    """
    # Generate public and private keys
    if force_private_key:
        x = CyclicInteger(force_private_key, q)
    else:
        x = CyclicInteger(random.randint(1, q-1), q)
    y = CyclicInteger(pow(g, int(x), p), p)
    
    # Random per message key (with r != 0)
    if nonce is None:
        k = CyclicInteger(random.randint(1, q-1), q)
    else:
        k = CyclicInteger(nonce, q)
    r = CyclicInteger(pow(g, int(k), p), q)
    while r == 0 and check_r_non_zero:
        k = CyclicInteger(random.randint(1, q-1), q)
        r = CyclicInteger(pow(g, int(k), p), q)
    
    # Calculate signature
    H = CyclicInteger(sha1i(text), q)
    s = k.inverse * (H + x*r) # These are all CyclicIntegers
    
    if s == 0:
        return dsa_sign(text, p, q, g) # Start again
    if return_private_key:
        return int(y), int(r), int(s), int(x), int(k)
    else:
        return int(y), int(r), int(s)

def dsa_verify(text, r, s, y, p, q, g):
    """Verify the signature of a given text with DSA
    """
    w = CyclicInteger(s, q).inverse
    H = CyclicInteger(sha1i(text), q)
    u1 = H * w
    u2 = CyclicInteger(r, q) * w
    v =((pow(g, int(u1), p) * pow(int(y), int(u2), p)) % p) % q
    return int(v) == int(r)

text = "Hello, world !"
y, r, s = dsa_sign(text, p, q, g)
assert dsa_verify(text, r, s, y, p, q, g)

In [15]:
# Let's first try to recover the private key from a test message
text = "What's wrong with the world mama ?"

def dsa_recover_private_key_from_nonce(text, r, s, y, p, q, g, k):
    """k is the nonce. The rest is simply a text with it's signature."""
    H = CyclicInteger(sha1i(text), q)
    r, s, k = CyclicInteger(r, q), CyclicInteger(s, q), CyclicInteger(k, q)
    return int(((s * k) - H) / r) # Cyclic Group Math ensured by the class

def dsa_check_private_key(text, r, s, y, p, q, g, k, x):
    """Let's see if a given recovered private key matches with the
    one that was actually used to generate a given signature.
    """
    y_test, r_test, s_test = dsa_sign(text, p, q, g, force_private_key=x, nonce=k)
    return y_test == y and r_test == r and s_test == s

y, r, s, x, k = dsa_sign(text, p, q, g, return_private_key=True)
x_recovered = dsa_recover_private_key_from_nonce(text, r, s, y, p, q, g, k)
assert x == x_recovered # Yay \o/
assert dsa_check_private_key(text, r, s, y, p, q, g, k, x_recovered)
assert not dsa_check_private_key(text, r, s, y, p, q, g, k, x_recovered + 1)

How this works (this is really trivial) :

$$\begin{align*}
((s.k) - H(msg)).r^{-1}
&= ((k^{-1}.(H(msg)+r.x).k)-H(msg)).r^{-1}\\
&= x
\end{align*}$$

In [16]:
# Now, the actual challenge.
# The author has used a nonce <2**16. This is really low and can easily be bruteforced.
y = 0x84ad4719d044495496a3201c8ff484feb45b962e7302e56a392aee4abab3e4bdebf2955b4736012f21a08084056b19bcd7fee56048e004e44984e2f411788efdc837a0d2e5abb7b555039fd243ac01f0fb2ed1dec568280ce678e931868d23eb095fde9d3779191b8c0299d6e07bbb283e6633451e535c45513b2d33c99ea17
r = 548099063082341131477253921760299949438196259240
s = 857042759984254168557880549501802188789837994940
text = """For those that envy a MC it can be hazardous to your health
So be friendly, a matter of life and death, just like a etch-a-sketch
"""
assert sha1i(text) == 0xd2d0714f014a9784047eaeccf956520045c45265

import tqdm
for k in tqdm.tqdm(range(2**16)):
    x_recovered = dsa_recover_private_key_from_nonce(text, r, s, y, p, q, g, k)
    if dsa_check_private_key(text, r, s, y, p, q, g, k, x_recovered):
        print('Found private key :', hex(x_recovered), ' (nonce : %s)' % k)
        break

assert sha1(hex(x_recovered)[2:]) == '0954edd5e0afe5542a4adf012611a91912a3ec16'

 25%|██▌       | 16539/65536 [00:17<00:52, 934.98it/s]

Found private key : 0x15fb2873d16b3e129ff76d0918fd7ada54659e49  (nonce : 16575)


### Exercice 44


DSA nonce recovery from repeated nonce

##### How it works

If k stays constant for two messages m1 and m2, then r stays constant too. This gives us (everything is mod q) :
$$\begin{align*}
\dfrac{m2-m1}{s2-s1}
&= (m2-m1).[k^{-1}.(m2+x.r)-k^{-1}.(m1+x.r)]^{-1}\\
&= k
\end{align*}$$

In [17]:
y = 0x2d026f4bf30195ede3a088da85e398ef869611d0f68f0713d51c9c1a3a26c95105d915e2d8cdf26d056b86b8a7b85519b1c23cc3ecdc6062650462e3063bd179c2a6581519f674a61f1d89a1fff27171ebc1b93d4dc57bceb7ae2430f98a6a4d83d8279ee65d71c1203d2c96d65ebbf7cce9d32971c3de5084cce04a2e147821

In [18]:
# Load into an object
messages = []
with open('44.txt') as _in:
    message = {}
    for l in _in:
        key = l.split(':')[0]
        value = l.split(':')[1].strip()
        if key == 'msg':
            value += ' ' # Apparently each message has a trailing space
        elif key == 'm':
            value = CyclicInteger(int(value, 16), q)
        else:
            value = CyclicInteger(int(value), q)
        message[key] = value
        if 's' in message and 'r' in message and 'msg' in message and 'm' in message:
            messages.append(message)
            message = {}

In [19]:
# Try to find messages with the same k.
# There is a neat trick here : if k is constant...
# ... r is constant too. Soooooo.... easy enough to
# identify the messages.
from itertools import combinations

m0, m1 = None, None
for m0, m1 in combinations(messages, 2):
    if m0['r'] == m1['r']:
        print('Found the two messages : {} and {}'.format(m0['msg'], m1['msg']))
        break

realfingerprint = 'ca8f6f7c66fa362d40760d135b763eb8527d3d52'
k = (m1['m'] - m0['m']) / (m1['s'] - m0['s'])
x_recovered = dsa_recover_private_key_from_nonce(m0['msg'], m0['r'], m0['s'], y, p, q, g, k)
x_recovered_2 = dsa_recover_private_key_from_nonce(m1['msg'], m1['r'], m1['s'], y, p, q, g, k)

assert dsa_check_private_key(m0['msg'], m0['r'], m0['s'], y, p, q, g, k, x_recovered)
assert dsa_check_private_key(m1['msg'], m1['r'], m1['s'], y, p, q, g, k, x_recovered)
assert sha1(hex(x_recovered)[2:]) == realfingerprint
assert sha1(hex(x_recovered_2)[2:]) == realfingerprint

Found the two messages : Listen for me, you better listen for me now.  and Pure black people mon is all I mon know. 


### Exercice 45

DSA parameter tampering

If we can force g = 0:
$$\begin{align*}
g=0
&=> r = 0\\
&=> s = k^{-1}.H(msg)\\
&=> y = 0
\end{align*}$$

This means the signature does no longer depend on the private key... Also, the nonce can be trivially extracted which is bad as we already know. Also the signature can be trivially forged (take the hash). In a word : completely broken.

In [20]:
text1, text2 = "Hello, world !", "How are you today ?"
y1, r1, s1 = dsa_sign(text1, p, q, g=0, check_r_non_zero=False)
assert y1 == r1 == 0
y2, r2, s2 = dsa_sign(text2, p, q, g=0, check_r_non_zero=False)
assert y2 == r2 == 0
h1 = sha1i(text1)
assert dsa_verify(text1, 0, h1, 0, p, q, g=0)

If we can force g = (p+1):
$$\begin{align*}
g=p+1
&=> r = g^{k}\ mod\ p\ =\ 1^{k}\ mod\ p\ =\ 1\\
&=> s = k^{-1}.(H(msg)+x)\\
&=> y = g^{x}\ mod\ p\ =\ 1^{k}\ mod\ p\ =\ 1\\
\end{align*}$$


In [21]:
text1 = "Hello, world !"
y1, r1, s1 = dsa_sign(text1, p, q, g=p+1)
print(y1, r1, s1)

z = sha1i("Goodbye")
r = ((y1 * z) % p) % q
s = (r * invmod(z, q)) % q
dsa_verify("text1", 1, 1, 1, p, q, g=p+1)

1 1 1370135020043054056608256218180985449163756199804


True

So, apparently, this is actually slightly more complicated and allows to generate a magic signature. However, I don't quite see where my math is wrong... So this is most definitely not as easy as that, but I can't seem to find the mistake.

### Exercice 46

RSA parity Oracle

In [22]:
# Use small primes for simplicity
p = 0xC2B0C8D9BFE064019955C4A3819E31BFA6679B32BC5A2021D1EDC5EA83BCCFD79946200A6D5C7D349B69B1BE3EE01AF3C48811D9B95BD8740BB5EA5B3BD1CAB5
q = 0xFC932034EA8B970B000C6C1CBBB7ACAB0ECC6D7C86840B75C9A2EAFC84536D04DF4F63B49B7852D65CC9FA313EB2E095C5A1B3E47BF9550447E65907741A43F9
r = RSA(p=p, q=q)

In [24]:
import base64
def oracle(encrypter, cypher):
    """Returns True if the plaintext is odd and False if it is even"""
    return bool(encrypter.decrypt_block(cypher, i2s=False) % 2)

message = "VGhhdCdzIHdoeSBJIGZvdW5kIHlvdSBkb24ndCBwbGF5IGFyb3VuZCB3aXRoIHRoZSBGdW5reSBDb2xkIE1lZGluYQ=="
message = base64.b64decode(message)
cypher = r.encrypt_block(message, i2s=False)

 25%|██▌       | 16539/65536 [00:30<01:28, 550.80it/s]

In [ ]:
import tqdm, math
lo, hi, c = 0, r.n, cypher
cypher_2 = r.encrypt(2, i2s=False)
pbar = tqdm.tqdm(range(int(math.log(r.n, 2))+20))
for i in pbar:
    pbar.set_description(str(int_to_str(hi)))
    if oracle(r, c*cypher_2):
        lo, hi, c = (hi+lo)//2, hi, c*cypher_2 - r.n
    else:
        lo, hi, c = lo, (hi+lo)//2, c*cypher_2

### Exercice 47

Bleichenbacher's PKCS 1.5 padding Oracle (Simple Case)

(moving directly to complete case)

### Exercice 48

Bleichenbacher's PKCS 1.5 padding Oracle (Complete Case)

(see PDF for explanation)